In [3]:
import pandas as pd
import numpy as np
import re

import plotly.express as px

In [4]:
df_raw = pd.read_csv('../Data/drug_data.csv', index_col=0)
df_raw.head()

,CASEID,DRUGID_1,CATID_1_1,CATID_2_1,CATID_3_1,ROUTE_1,TOXTEST_1,sdled_1_1,sdled_2_1,sdled_3_1,...,QUARTER,DAYPART,NUMSUBS,CASETYPE,DISPOSITION,ALLABUSE,METRO,AGECAT,SEX,RACE
0,1,865,105,110,2005,1,2,1,2.0,-7.00,...,4,1,2,8,6,1,2,4,1,2
1,2,2077,81,82,283,-8,2,17,67.0,67.01,...,3,4,1,4,7,0,2,11,1,3
2,3,2313,1,12,-7,-8,2,17,64.0,64.99,...,4,3,1,4,1,0,7,11,2,2
3,4,234,358,99,215,2,2,17,73.0,73.01,...,2,2,1,4,7,0,10,2,1,3
4,5,865,105,110,2005,1,2,1,2.0,-7.00,...,3,4,3,8,8,1,1,6,1,3


In [20]:
df_raw.columns

Index(['CASEID', 'DRUGID_1', 'CATID_1_1', 'CATID_2_1', 'CATID_3_1', 'ROUTE_1',
       'TOXTEST_1', 'sdled_1_1', 'sdled_2_1', 'sdled_3_1', 'sdled_4_1',
       'sdled_5_1', 'sdled_6_1', 'DRUGID_2', 'CATID_1_2', 'CATID_2_2',
       'CATID_3_2', 'ROUTE_2', 'TOXTEST_2', 'sdled_1_2', 'sdled_2_2',
       'sdled_3_2', 'sdled_4_2', 'sdled_5_2', 'sdled_6_2', 'DRUGID_3',
       'CATID_1_3', 'CATID_2_3', 'CATID_3_3', 'ROUTE_3', 'TOXTEST_3',
       'sdled_1_3', 'sdled_2_3', 'sdled_3_3', 'sdled_4_3', 'sdled_5_3',
       'sdled_6_3', 'ALCOHOL', 'NONALCILL', 'PHARMA', 'NONMEDPHARMA',
       'CASEWGT', 'YEAR', 'QUARTER', 'DAYPART', 'NUMSUBS', 'CASETYPE',
       'DISPOSITION', 'ALLABUSE', 'METRO', 'AGECAT', 'SEX', 'RACE'],
      dtype='object')

In [5]:
casetype_dummies = pd.get_dummies(df_raw['CASETYPE'])\
    .rename(columns={1: 'SUICIDE', 2: 'SEEKING_DETOX'})

In [6]:
columns_to_drop = ['CASEID', 'YEAR', 'QUARTER', 'DAYPART', 'NUMSUBS', 'DISPOSITION']
df_drop_unwanted = df_raw.drop(columns=columns_to_drop)

In [9]:
df_abuse_indicators = pd.concat([df_drop_unwanted, casetype_dummies.iloc[:, :2]], axis=1)

In [11]:
df_abuse_indicators.to_csv('../Data/abuse_indicator_data.csv')

In [10]:
df_abuse_indicators.head()

,DRUGID_1,CATID_1_1,CATID_2_1,CATID_3_1,ROUTE_1,TOXTEST_1,sdled_1_1,sdled_2_1,sdled_3_1,sdled_4_1,...,NONMEDPHARMA,CASEWGT,CASETYPE,ALLABUSE,METRO,AGECAT,SEX,RACE,SUICIDE,SEEKING_DETOX
0,865,105,110,2005,1,2,1,2.0,-7.00,-7.0000,...,0,0.942635,8,1,2,4,1,2,0,0
1,2077,81,82,283,-8,2,17,67.0,67.01,67.0140,...,0,5.992011,4,0,2,11,1,3,0,0
2,2313,1,12,-7,-8,2,17,64.0,64.99,-7.0000,...,0,4.723172,4,0,7,11,2,2,0,0
3,234,358,99,215,2,2,17,73.0,73.01,73.0106,...,0,4.080147,4,0,10,2,1,3,0,0
4,865,105,110,2005,1,2,1,2.0,-7.00,-7.0000,...,0,5.177709,8,1,1,6,1,3,0,0
